# D.4 Lagrangian Calculations

In [1]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

### Define Symbols and Configuration Variables

In [2]:
# Time symbol
t = symbols('t')

In [3]:
# Generalized coordinates

# TODO: Change this!
z = symbols('z', cls=Function)
z = z(t)

In [4]:
# Derivatives of Generalized Coordinates

# TODO: Change this!
z_dot = z.diff(t)
z_ddot = z_dot.diff(t)

In [5]:
# Other symbols (mass, forces, damping coefficients, torques...)

# TODO: Change this!
k, b, m, F  = symbols('k, b, m, F')

### Kinetic Energy

#### Translational Kinetic Energy

In [6]:
# TODO: Change this!
K_T = 1/2*m*z_dot**2

dotprint(K_T)

<IPython.core.display.Math object>

#### Rotational Kinetic Energy

In [7]:
# TODO: Change this!
K_R = 0

dotprint(K_R)

<IPython.core.display.Math object>

#### Total Kinetic Energy

In [8]:
K = K_T #+ K_R

dotprint(K)

<IPython.core.display.Math object>

### Potential Energy

In [9]:
# TODO: Change this!
P = 1/2*k*z**2

dotprint(P)

<IPython.core.display.Math object>

### Define the Lagrangian

For this class, the Lagrangian will in general be a scalar value equal to the difference between kinetic and potential energy.

In [10]:
L = K - P

dotprint(L)

<IPython.core.display.Math object>

### Define the Forces

#### Generalized Forces

The forces will form a vector with the same length as the number of configuration variables. 

The first entry corresponds to the first configuration variable, the second entry to the second, etc.

Each entry should contain the generalized forces acting on the corresponding configuration variable.

In [11]:
# TODO: Change this!
tau = Matrix([F])

dotprint(tau)

<IPython.core.display.Math object>

#### Non-conservative Forces (e.g. Damping)

Similar to the generalized forces, this will be a vector with the same length as the number of configuration variables.

Each entry should contain the nonconservative forces acting on the corresponding configuration variable.

In [12]:
# TODO: Change this!
B = Matrix([-b])
q_dot = Matrix([z_dot])

Bqdot = B @ q_dot

dotprint(Bqdot)

<IPython.core.display.Math object>

In [13]:
F_total = tau + Bqdot

dotprint(F_total)

<IPython.core.display.Math object>

### Compute the Lagrangian Derivatives / Partial Derivatives

The following function computes these two terms:

* $\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right)$
* $\frac{\partial L(q,\dot{q})}{\partial q}$

Things to note:

* `q` and `q_dot` should be Python tuples containing your configuration variables (e.g. `(x, theta)`)
* The function returns a Python list. You probably want to wrap the output in a Sympy `Matrix()`. For example,

```python
out = derive_Lagrangian(...)
out = Matrix(out)
```

In [14]:
def derive_Lagrangian(L, q, q_dot):
    term_1 = (sympy.tensor.derive_by_array(L, q_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, q)
    return term_1 - term_2

### Get the final Euler Lagrange Equations of Motion

Remember that the Euler Lagrange Equations used in class are:

$$
\underbrace{\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right) - \frac{\partial L(q,\dot{q})}{\partial q}}_\text{Left Hand Side (LHS)} = \underbrace{\tau - B \dot{q}}_\text{Right Hand Side (RHS)}.
$$

In Sympy, we use the `Eq(LHS, RHS)` class to get a symbolic equation $\text{LHS} = \text{RHS}$.

Both the left and right expressions should be Sympy vectors (i.e. Matrices with 1 column). 

The length of the left and right vectors must both be equal to the number of configuration variables.

In [15]:
# TODO: Change this!
LHS = Matrix([derive_Lagrangian(L, z, z_dot)])
RHS = F_total

dynamics = Eq(LHS, RHS)

dotprint(dynamics)

<IPython.core.display.Math object>

# D.4 Start

In [16]:
solve_dynamics = solve(dynamics, (z_ddot,), simplify = True, dict = True)[0]
dotprint(solve_dynamics)

<IPython.core.display.Math object>

In [28]:
state = MatrixSymbol('x', 2,1)

In [18]:
z_ddot_expr = solve_dynamics[z_ddot]
state_deriv = Matrix([z_dot, z_ddot_expr])
dotprint(state_deriv)

<IPython.core.display.Math object>

In [19]:
subs_dict = {
    z: state[0],
    z_dot: state[1]
}

In [20]:
f_expr = state_deriv.subs(subs_dict)
dotprint(f_expr)

<IPython.core.display.Math object>

finding equilibrium things

In [21]:
equilibrium_equation = Eq(f_expr, Matrix([0,0]))

eq_solve_dict = solve(equilibrium_equation, (state[0], state[1], F), simplify=True, dict=True)[0]
dotprint(eq_solve_dict)

<IPython.core.display.Math object>

In [22]:
u_eq = eq_solve_dict[state[1]]
z_dot_eq = eq_solve_dict[state[0]]

In [23]:
dotprint(f_expr.subs({state[0]: 0, state[1]: 0}))

<IPython.core.display.Math object>

# jacobians...

In [24]:
A = f_expr.jacobian(state)
dotprint(A)

<IPython.core.display.Math object>

In [25]:
z_desired = F/m

A_eq = A.subs({state[0]: 0, state[1]: 0})
dotprint(A_eq)

<IPython.core.display.Math object>

In [26]:
B = f_expr.jacobian([F])
dotprint(B)

<IPython.core.display.Math object>

# H = Ax + Bu

In [27]:
H = A @ Matrix([z, z_dot]) - B @ Matrix([tau])
dotprint(H)

<IPython.core.display.Math object>